<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/post_ollama_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import json
import os
import time
from tqdm import tqdm

In [20]:
#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          ##################################     96.9%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [21]:
#un thread demone avvia il server locale di ollama
import subprocess
import threading
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
t.start()

In [22]:
#un altro thread demone avvia llama3
!ollama pull llama3
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3"]),daemon=True)
t2.start()

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJoc4Nm1Nj9/D2IOMuUNEfAqdxcMNB3WgqjVSWtiEere



2024/07/02 15:42:02 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-07-02T15:42:02.473Z level=INFO source=images.go:730 msg="total blobs: 0"
time=2024-07-02T15:42:02.474Z level=INFO source=images.go:737 msg="total unused blobs removed: 0"
time=2024-0

[GIN] 2024/07/02 - 15:42:07 | 200 |      77.305µs |       127.0.0.1 | HEAD     "/"


time=2024-07-02T15:42:07.792Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60101]"
time=2024-07-02T15:42:07.944Z level=INFO source=types.go:98 msg="inference compute" id=GPU-9084be9d-a939-fa5f-4f90-7fcd227b4be7 library=cuda compute=6.0 driver=12.2 name="Tesla P100-PCIE-16GB" total="15.9 GiB" available="15.6 GiB"


pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  

...

pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c...   0% ▕                ▏    0 B/ 254 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c...   0% ▕                ▏    0 B/ 254 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c...   0% ▕                ▏    0 B/ 254 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c...   0% ▕                ▏    

time=2024-07-02T15:42:45.532Z level=INFO source=download.go:136 msg="downloading 577073ffcc6c in 1 110 B part(s)"


pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c...   0% ▕                ▏    0 B/ 110 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c...   0% ▕                ▏    0 B/ 110 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c...   0% ▕                ▏    0 B/ 110 B       

time=2024-07-02T15:42:48.749Z level=INFO source=download.go:136 msg="downloading 3f8eb4da87fa in 1 485 B part(s)"


pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa...   0% ▕                ▏    0 B/ 485 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa...   0% ▕                ▏    0 B/ 485 B                  pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB              

In [23]:
moderator_instructions = [
'You are a decentralized social-network instance\'s moderator. Your role is to check if utent\'s post respects the instance rules. You must assign each post a score from 0 (if the post violate completely the instance\'s rules) to 10 (if the post respects all the instance\'s rules). Your response must be only the score number and anything else.' ,
'You are a decentralized social-network instance\'s owner. Your role is to check if utent\'s post respects the instance rules. In order to classify posts you have to assign each post a score from 0(if the post violate completely the instance\'s rules) to 10(if the post respects all the instance\'s rules). You must be very strict in your evaluations. Please pay a lot of attention and let\'s think step by step. Your response must be only the score number and anything else.'
]


In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory, file_name):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [24]:
def ask_to_llama(instruction,rules, prompt):    
    full_context = instruction
    for i, rule in enumerate(rules, start=1):
        full_context += f"{i}. {rule['text']}\n"
    
    full_prompt = full_context + prompt
    
    response = requests.post('http://localhost:11434/api/generate', 
                             data=json.dumps({'model': 'llama3', 'prompt': full_prompt, 'stream': False}), 
                             headers={'Content-Type': 'application/json'})
    
    return response.json()['response']

[GIN] 2024/07/02 - 15:43:04 | 200 |      36.102µs |       127.0.0.1 | HEAD     "/"


In [25]:
def process_json_files(instances, start, end, save_dir):
    
    start_time = time.time()
    istanze = os.listdir(instances)
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    already_proc = os.listdir(save_dir) + os.listdir('/kaggle/input/instance-json/scores/scores')
    
    count = 0
    for i in tqdm(range(start, end)):
        nome=f'scores_firefox_{i}.zip'
        filename = istanze[i]
        if not f'{filename[:-5]}_scores.json' in already_proc:
            count+=1
            if (count%1)==0:
                zip_dir(save_dir,nome)
            filepath = os.path.join(instances, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                #print("DATA: ",data)
                instance_name = data['name']
                results = {'name': instance_name, 'rules': data['rules'], 'records':[]}
                rules = data['rules']
                
                records = data['records']
                records_en = [record for record in records if record['language'] == 'en']
                        
                if len(records_en)==0:
                    print(f"Per l'istanza {instance_name} non ci sono post in inglese")
                    filt_recorods = []
                    
                if len(records_en)>200: #se len>200 prendiamo solo testa e coda
                    records_ordinati = sorted(records_en, key=lambda x: x['favourites'])
                    primi_100 = records_ordinati[:100]
                    ultimi_100 = records_ordinati[-100:]
                    filt_records = primi_100 + ultimi_100
                    
                else: #altrimenti li prendiamo tutti
                    filt_records = records_en
                
                #print("Numero di post: ",len(filt_records))
                i=0
                if len(filt_records)>0 and isinstance(rules, list):
                    for record in filt_records:
                        record_text = record['text']
                        rec = {'id': record['id'], 'text': record_text, 'tests':[]}
                        i+=1
                        for instruction in moderator_instructions:
                            response = ask_to_llama(instruction,rules,record_text)
                            rec['tests'].append({'instruction': instruction, 'score': response})
                            #break #così vediamo cosa fa con le altre instanze
                        results['records'].append(rec)
                        print(f"Processati {i} post")
                    print(f'instance {instance_name} successfully processed\n')
                    with open(os.path.join(save_dir, f'{instance_name}_scores.json'),'w') as f:
                        json.dump(results, f, indent = 4)
                    
    end_time = time.time() 
    duration = end_time - start_time
    print("DURATA: ", duration)

In [ ]:
da = 550
a = 660
instances = '/kaggle/input/instance-json/results/results'
#print(os.listdir(instances))
process_json_files(instances,da,a,'/kaggle/working/scores')
#len(os.listdir(instances))//num_profili


[GIN] 2024/07/02 - 15:43:04 | 200 |   37.284483ms |       127.0.0.1 | POST     "/api/show"


  0%|          | 0/110 [00:00<?, ?it/s]

INFO [main] build info | build=1 commit="7c26775" tid="139304778457088" timestamp=1719934985


INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="139304778457088" timestamp=1719934985 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="3" port="43853" tid="139304778457088" timestamp=1719934985


⠙ time=2024-07-02T15:43:05.072Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.required.full="5.0 GiB" memory.required.partial="5.0 GiB" memory.required.kv="256.0 MiB" memory.required.allocations="[5.0 GiB]" memory.weights.total="3.9 GiB" memory.weights.repeating="3.5 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="164.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-07-02T15:43:05.072Z level=INFO source=server.go:368 msg="starting llama server" cmd="/tmp/ollama406712023/runners/cuda_v11/ollama_llama_server --model /root/.ollama/models/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa --ctx-size 2048 --batch-size 512 --embedding --log-disable --n-gpu-layers 33 --parallel 1 --port 43853"
time=2024-07-02T15:43:05.073Z level=INFO source=sched.go:382 msg="loaded runners" count=1
time=2024-07-02T15:43:05.073Z level=INFO source=server.

INFO [main] model loaded | tid="139304778457088" timestamp=1719935028


time=2024-07-02T15:43:48.773Z level=INFO source=server.go:599 msg="llama runner started in 43.70 seconds"


[GIN] 2024/07/02 - 15:43:48 | 200 |  43.83597474s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/02 - 15:43:49 | 200 | 43.759246024s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/02 - 15:43:50 | 200 |   1.06467071s |       127.0.0.1 | POST     "/api/generate"
Processati 1 post
[GIN] 2024/07/02 - 15:43:51 | 200 |  862.093605ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/02 - 15:43:52 | 200 |  1.057934799s |       127.0.0.1 | POST     "/api/generate"
Processati 2 post
[GIN] 2024/07/02 - 15:43:53 | 200 |  1.060963866s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/07/02 - 15:43:54 | 200 |  1.067061079s |       127.0.0.1 | POST     "/api/generate"
Processati 3 post
[GIN] 2024/07/02 - 15:43:55 | 200 |  1.060879191s |       127.0.0.1 | POST     "/api/generate"


In [1]:
!zip -r scores_firefox.zip /kaggle/working/scores

	zip warning: name not matched: /kaggle/working/scores

zip error: Nothing to do! (try: zip -r scores_firefox.zip . -i /kaggle/working/scores)
